In [10]:
import re
import uuid
from openai import AzureOpenAI
from pinecone import Pinecone, ServerlessSpec
from langchain_openai import AzureOpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

In [6]:
import os
import time
# Set an environment variable
os.environ['PINECONE_API_KEY'] = '193bfd5b-1a4a-4c8b-bc06-7ec7c4cfc66a'

PINECONE_API_KEY="193bfd5b-1a4a-4c8b-bc06-7ec7c4cfc66a"

In [7]:
pc = Pinecone(api_key=PINECONE_API_KEY)

In [8]:
index_name="indexdb"

In [14]:
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

In [11]:
gpt_engine_name="gpt-35-turbo-16k"
embedding_model_name="text-embedding-002"
azure_endpoint = "https://securityappdewa.openai.azure.com"
api_key="576b185ff87d4f50962560d349d06332"
api_version= "2023-07-01-preview"

In [12]:
embeddings = AzureOpenAIEmbeddings(
            model=embedding_model_name, 
            api_key=api_key, 
            azure_endpoint=azure_endpoint, 
            openai_api_version=api_version)

In [13]:
docsearch = PineconeVectorStore(index_name=index_name, embedding=embeddings)

In [48]:
query="what is document retriveal"
filters={
  "filename": { "$in": ['djkfkjkd'] }
}

In [50]:
len(filters['filename']['$in'])

1

In [45]:
if filters['filename']['$in']:
    print('ok')
else:
    print('ok1')

ok1


In [33]:
docs = docsearch.similarity_search(query, k=100)

In [34]:
docs

[Document(metadata={'filename': 'kmbot.pdf', 'page_no': 6.0}, page_content='\x7f- Access control policies\n\x7f- User authentication and authorization\n\x7fUser Experience:\n\x7fLogin requirement: Users will be required to login to access the Document QnA system.\n\x7fAuthorization Requirements:\n\x7fThe system will have different roles and permissions, such as:\n\x7f- Coordinator\n\x7f- Manager\n\x7f- Senior Management\n\x7f- Higher Management\n\x7fSMS/Email Content:\n\x7fThe Document QnA system may require sending SMS or email notifications/communications, the content of which\nwill be defined during the project execution.\nProcess\nThe process described in this section focuses on document retrieval, question submission, answer generation,\nand user-based permissions within the Dubai Electricity & Water Authority (DEWA) system. These processes\nare crucial for effective project management and efficient service delivery. The following steps outline the\nprocess flow and key considerat

In [1]:
%pip install camelot-py[cv] ghostscript

You should consider upgrading via the 'c:\Users\DudukuntaSashidharRe\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import camelot

# Path to the PDF file
pdf_file = "financial_document.pdf"

# Extract tables from the PDF
tables = camelot.read_pdf(pdf_file, pages='all')

# Print the number of tables found
print(f"Total tables found: {len(tables)}")

# Iterate over the tables and print the first few rows of each
for i, table in enumerate(tables):
    print(f"\nTable {i + 1}:")
    print(table.df.head())

    # Optionally, you can save each table as a CSV file
    table.to_csv(f"table_{i + 1}.csv")

# If you want to export all tables in one CSV file, uncomment the following line:
# tables.export('tables.csv', f='csv')


DeprecationError: PdfFileReader is deprecated and was removed in PyPDF2 3.0.0. Use PdfReader instead.

In [96]:
system_prompt='''You are an intent classification agent. Your task is to identify the intent of the provided user query. Carefully analyze the query and classify it into one of the following intents.

Intents Definitions:
graph: If the user query explicitly requests the generation of any type of graph from data, classify the intent of the query as "graph".
summary: If a user query 'explicitly ask for a summary' of a topic or document, classify the intent of that query as "summary". Only classify it as summary intent if the summary word is explicitly mentioned in the user query.
normal: If the user query is a general question using words like "where," "what," "tell," etc., and does not fit into the "summary" or "graph" intents, classify the intent as "other."

Carefully analyze the given user query along with the above intent definitions provided, and classify the query as either "graph", "summary" or "normal" based on its content.

Output only one of the following intents: graph, summary or normal. No additional explanation or information is allowed.
'''

In [97]:
user_prompt='''Based on the user query provided, classify its intent. Analyze the intent definitions carefully before determining whether the intent is "summary" "graph" or "normal"
Only classify it as summary intent if the summary or summarize words are 'explicitly mentioned' in the user query.
Output only one of the following intents: summary, graph, or normal.

user query:
{query}

Intent:
'''

In [142]:
gpt_engine_name="gpt-35-turbo-16k"
embedding_model_name="text-embedding-002"
azure_endpoint = "https://securityappdewa.openai.azure.com"
api_key="576b185ff87d4f50962560d349d06332"
api_version= "2023-07-01-preview"

In [99]:
openai_client = AzureOpenAI(
                    azure_endpoint = azure_endpoint,
                    api_key=api_key,
                    api_version= api_version
                )

In [126]:
query="what is acceleration motion"

In [127]:
res = openai_client.chat.completions.create(model=gpt_engine_name,
                                                        messages=[
                                                                {'role': 'system','content':system_prompt},  
                                                                {'role': 'user',  'content':f'''Based on the user query provided, classify its intent. Analyze the intent definitions carefully before determining whether the intent is "summary" "graph" or "normal"
Only classify it as summary intent if the summary or summarize words are 'explicitly mentioned' in the user query. Strictly check for words like "summarize," "summarise," and similar terms, and classify the intent as "summary" only if these words are present in the user query.
Output only one of the following intents: graph, summary, or normal.

user query:
{query}

Think step by step and identify the intent of above user query.

Intent:
'''}],
                                                        temperature=0.1,
                                                        max_tokens=1
                                                        )

In [128]:
res.choices[0].message.content

'normal'

In [35]:
import pandas as pd
from io import StringIO

In [71]:
df=pd.read_excel("POP.xlsx")

In [73]:
df.dtypes

Code          object
Rank           int64
Country       object
Population     int64
Unnamed: 4    object
dtype: object

In [72]:
exec('''import plotly.graph_objects as go

# Select the relevant columns for the graph
selected_columns = ['Country', 'Population']

# Sort the dataframe based on population in descending order
df_sorted = df.sort_values(by='Population', ascending=False)

# Select the top 7 countries based on population
df_top_7 = df_sorted.head(7)

# Create the graph
fig = go.Figure(data=go.Bar(
    x=df_top_7['Country'],
    y=df_top_7['Population']
))

# Set the x and y axis labels
fig.update_layout(
    xaxis=dict(title='Country'),
    yaxis=dict(title='Population')
)

# Set the title of the graph
fig.update_layout(title='Top 7 Population Countries')

# Set the size of the graph
fig.update_layout(
    height=700,
    width=900
)

# Save the graph as html
fig.show()''')

In [50]:
df.columns

Index(['Unnamed: 0', 'EEID', 'Full Name', 'Job Title', 'Department',
       'Business Unit', 'Gender', 'Ethnicity', 'Age', 'Hire Date',
       'Annual Salary', 'Bonus %', 'Country', 'City', 'Exit Date'],
      dtype='object')

In [51]:
mf=df.to_markdown(index=False)

In [57]:
# Convert markdown to a DataFrame
data = pd.read_csv(StringIO(mf), sep="|")

In [58]:
data

,Unnamed: 0,Unnamed: 0,EEID,Full Name,Job Title,Department,Business Unit,Gender,Ethnicity,Age,Hire Date,Annual Salary,Bonus %,Country,City,Exit Date,Unnamed: 16
0,NaN,-------------:,:-------,:---------------------,:-------------------------------,:----------------,:-----------------------,:---------,:------------,------:,:--------------------,----------------:,----------:,:--------------,:---------------,:--------------------,NaN
1,NaN,0,E02387,Emily Davis,Sr. Manger,IT,Research & Development,Female,Black,55,2016-04-08 00:00:00,141604,0.15,United States,Seattle,2021-10-16 00:00:00,NaN
2,NaN,1,E04105,Theodore Dinh,Technical Architect,IT,Manufacturing,Male,Asian,59,1997-11-29 00:00:00,99975,0,China,Chongqing,NaT,NaN
3,NaN,2,E02572,Luna Sanders,Director,Finance,Speciality Products,Female,Caucasian,50,2006-10-26 00:00:00,163099,0.2,United States,Chicago,NaT,NaN
4,NaN,3,E02832,Penelope Jordan,Computer Systems Manager,IT,Manufacturing,Female,Caucasian,26,2019-09-27 00:00:00,84913,0.07,United States,Chicago,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,NaN,995,E03094,Wesley Young,Sr. Analyst,Marketing,Speciality Products,Male,Caucasian,33,2016-09-18 00:00:00,98427,0,United States,Columbus,NaT,NaN
997,NaN,996,E01909,Lillian Khan,Analyst,Finance,Speciality Products,Female,Asian,44,2010-05-31 00:00:00,47387,0,China,Chengdu,2018-01-08 00:00:00,NaN
998,NaN,997,E04398,Oliver Yang,Director,Marketing,Speciality Products,Male,Asian,31,2019-06-10 00:00:00,176710,0.15,United States,Miami,NaT,NaN
999,NaN,998,E02521,Lily Nguyen,Sr. Analyst,Finance,Speciality Products,Female,Asian,33,2012-01-28 00:00:00,95960,0,China,Chengdu,NaT,NaN


In [53]:
# Clean up extra whitespace in columns
data = data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [54]:
data = data.drop(columns=data.columns[[0, -1]])

In [55]:
data = data.drop(index=0)

In [60]:
data.columns = data.columns.str.strip()

In [61]:
data.columns

Index(['Unnamed: 0', 'Unnamed: 0', 'EEID', 'Full Name', 'Job Title',
       'Department', 'Business Unit', 'Gender', 'Ethnicity', 'Age',
       'Hire Date', 'Annual Salary', 'Bonus %', 'Country', 'City', 'Exit Date',
       'Unnamed: 16'],
      dtype='object')

In [64]:
exec('''
import plotly.graph_objects as go

# Selecting the relevant columns for the graph
df_selected = df[['Country', 'Population']].head(7)

# Sorting the data in descending order based on population
df_selected_sorted = df_selected.sort_values(by='Population', ascending=False)

# Creating the bar graph
fig = go.Figure(data=go.Bar(x=df_selected_sorted['Country'],
                           y=df_selected_sorted['Population'],
                           marker_color='blue'))

# Adding labels and title to the graph
fig.update_layout(title='Population in Top 7 Countries',
                  xaxis_title='Country',
                  yaxis_title='Population')

# Saving the graph in html format
fig.write_html('temp/graph_ca8f6973581e11ef8c84fff0fd93fc55.html')
''')

In [161]:
python_prompt='''You are a senior python developer. Your task is to generate precise Plotly Python code to create visual graphs based on the provided dataframe sample data, dataframe name, dataframe columns, and user query.
Carefully analyse the given dataframe name, dataframe sample data, user query to write accurate python code.

Important points to note before generating code:
- The dataframe is already loaded in memory, so just use the given dataframe name in code. Never generate sample data just use the dataframe name.
- The graph should have accurate x_axis, y_axis labels with title. If the graph is stacked then show it in different colors with legend labels. The labels of the graph is very important. Title of the graph should be there at top or bottom of the graph.
- The minimum size of graph should be of height 700 and width 900.
- If the user asking about yearwise or quarterwise or monthwise data then show respective data only in the graph without extra information.
- In the generated python code, save the graph in html format to this path - 'temp/graph_{id}.html'

Follow the below steps to generate accurate python code for user query.

Step 1: Exhaustively analyse the given dataframe name, dataframe sample data, user query to write accurate python code.

Step 2: Based on the analysis in Step 1, select the type of graph that is most suitable for the given dataframe sample data according to the user query. Additionally, determine which columns are most relevant and should be considered for generating the graph as per the user query.
Only use the below available dataframe columns in code.
dataframe columns:
{columns}

Step 3: Select the most appropriate x-axis and y-axis from the dataframe based on the user query. These axes should clearly address the query and contain relevant values with appropriate labels.

Step 4: Based on the choices made in Steps 2 and 3, including the graph type, x-axis, and y-axis, think through the process step-by-step to generate accurate Python code that will create the desired graph using the given dataframe sample data, dataframe name, and user query.

Follow all the above steps and important points to generate precise python code.
'''

In [162]:
def excecute_python_code(df, query, python_prompt):
    # if gpt_access=='gpt-4':
    #     gpt_engine_name=gpt4_model_name
    # else:
    #     gpt_engine_name=gpt_model_name
    try:
        id=uuid.uuid1().hex
        col=df.columns.to_list()
        python_prompt=python_prompt.format(id=id, columns=col)
        messages=[{'role':'system', 'content':python_prompt}]
        messages.append({'role': 'user', 'content': f'''Your task is to generate Python code using the Plotly library to create a graph based on the provided dataframe sample data, dataframe name, and user query. Use only the available dataframe columns in creating the graph.
The dataframe is already loaded in memory, so just use the given dataframe name in code. Never read the dataframe, just use the dataframe name in code as it is already loaded in memory.

Dataframe Name:
df
                
Dataframe sample data:
{df.head}
                        
user query:
{query}

Do not generate any explanation for code just generate python code only.

Python Code:
'''})
        ans = openai_client.chat.completions.create(
                            model=gpt_engine_name,
                            messages=messages,
                        )
        res=ans.choices[0].message.content
        try:
            pattern = r"```python(.*?)```"
            matches = re.findall(pattern, res, re.DOTALL)
            if matches:
                python_code = matches[0].strip()
            else:
                raise
        except Exception as e:
            print("code parsing error occured")
            pattern = r'```(.*?)```'
            matches = re.findall(pattern, res, re.DOTALL)
            if matches:
                python_code = matches[0].strip()
            else:
                python_code=res
        try:
            print('##################')
            exec(python_code)
        except Exception as e:
            error=e
            print('code excecution error', e)
            retry_limit=2
            count=0
            messages=[
                    {'role': 'system', 'content': f'''You are a senior python developer. Your task is to correct the errors in the python code based on error message, dataframe name, dataframe sample data and user query. The python code is to generate visual graphs using plotly. Based on the given pyhton code and error message generate correct python code.
The dataframe is already loaded in memory, so just use the given dataframe name in code. Never generate sample data just use the dataframe name.'''}
                    ]
            while count<retry_limit:
                print("@@@@@@@@@@@", count)
                messages.append({'role': 'user', 'content': f'''Your task is to correct the given Python code using the Plotly library to create a graph based on the provided error message, dataframe sample data, dataframe name, python code, and user query. Use only the available dataframe columns in creating the graph.
Only use the dataframe name to create the graph. Only use the below dataframe name, dataframe sample data and user query in correcting code.

Dataframe Name:
df
                
Dataframe sample data:
{df.head}

python code:
{python_code}

error message:
{error}
                        
user query:
{query}

Do not generate any explanation for code just correct the above wrongly generated python code.

Correct Python Code:
'''})
                response = openai_client.chat.completions.create(
                            model=gpt_engine_name,
                            messages=messages
                        )
                res = response.choices[0].message.content
                try:
                    pattern = r"```python(.*?)```"
                    matches = re.findall(pattern, res, re.DOTALL)
                    if matches:
                        python_code = matches[0].strip()
                    else:
                        raise
                except Exception as e:
                    print("code parsing error occured")
                    pattern = r'```(.*?)```'
                    matches = re.findall(pattern, res, re.DOTALL)
                    if matches:
                        python_code = matches[0].strip()
                    else:
                        python_code=res
                try:
                    exec(python_code)
                    break
                except Exception as e:
                    error=e
                count+=1
                messages.append({"role": "assistant", "content": python_code})
        print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
        print(python_code)
    except Exception as e:
        print("Error in excecute_python_code", e)

In [163]:
query="bar graph showing count employees in different department"

In [164]:
excecute_python_code(df, query, python_prompt)

code parsing error occured
##################
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
import plotly.graph_objects as go

# Calculate the count of employees in each department
department_count = df['Department'].value_counts()

# Create a bar graph
fig = go.Figure(data=go.Bar(x=department_count.index, y=department_count.values))

# Customize the graph
fig.update_layout(
    title="Count of Employees in Different Departments",
    xaxis_title="Department",
    yaxis_title="Number of Employees",
    width=900,
    height=700
)

# Save the graph as an HTML file
fig.write_html("temp/graph_cab15ef457e411ef9ce1fff0fd93fc55.html")


In [146]:
import plotly.express as px

# Selecting relevant columns for the graph
df_selected = df[['Department', 'Full Name']].copy()

# Grouping data by department and counting the number of employees in each department
df_grouped = df_selected.groupby('Department').count().reset_index()

# Creating the bar graph
fig = px.bar(df_grouped, x='Department', y='Full Name', color='Department')

# Adding labels and title to the graph
fig.update_layout(
    xaxis_title='Department',
    yaxis_title='Number of Employees',
    title='Employees in Different Departments'
)

# Saving the graph as html file
fig.show()

In [1]:
text='''Baroda Home Loan
Are you are dreaming of buying your own residence? Opt for Bank of Baroda Home 
Loan. Our bob home loan is packed with several exclusive features and benefits for 
aspiring homeowners. Our housing loan can be used for a variety of purposes. For 
instance, you can buy a plot, purchase a flat, build your own home and even extend 
your existing residence with a home loan. Use the bob home loan calculator to find 
out bob home loan interest rate & more!
Benefits:
 Low Interest rates
 Low processing charges
 Higher loan amount
 Free Credit card
 Longer tenures\
Features
 Home Loan products to suit every customers need
 The approved home loan amount varies according to location and income of 
the applicants.
 Low Interest Rates
 Low Processing Fee
 No Hidden Charges
 No Pre Payment Penalty
 The interest rate is linked to Baroda Repo Linked Lending Rate (BRLLR) of 
the bank and is reset monthly.
 Interest charges on Daily Reducing Balance
 Repayment up to 30 years
 Moratorium period up to 36 months after the loan amount is disbursed.
 Generally, we accept mortgages of the constructed or purchased property as 
collateral. In some cases, collateral for home loans can be accepted in the 
form of insurance policies, government promissory notes, shares and 
debentures, gold ornaments and other property etc.
Eligibility Criteria
Resident Type:
 Resident Indians
 Non-Resident Indians (NRIs) holding Indian passport or Persons of 
Indian origin (PIOs) holding foreign passport or Overseas Citizens of 
India (OCI)
Minimum Age:
 Applicant:- 21 Years
 Co-applicant:- 18 Years
 Maximum Age: 70 years
 Loan Tenure: up to 30 years.
Maximum Loan Amount
 Mumbai : Rs. 20 Crores
 Hyderabad, New Delhi (including National Capital Region) and 
Bengaluru: Rs.7.50 Crore
 Other Metros : Rs. 5.00 Crores
 Urban Areas : Rs. 3.00 Crores
 Semi-urban and Rural : Rs. 1.00 Crores
 Chandigarh, Panchkula & Mohali:- Rs. 5 Crores
Baroda Home Loan : Interest rates & charges
Floating Rate:
Product Conditions
Repo Rate + 
Spread
Effective Rate of 
Interest
Baroda Home 
Loan to NonStaff members
For Salaried* BRLLR - 0.75% 
to BRLLR +
1.45%
From 8.40% to 
10.60%
For NonSalaried*
BRLLR - 0.75% 
to BRLLR + 
1.45%
From 8.40% to 
10.60%
* Note:
 Rate of Interest is dependent upon Loan limit and CIBIL Score of the 
applicant/s.
 Risk Premium of 0.05% will be levied for customers not obtaining credit 
insurance cover
Fixed Rate:
Product Conditions
Repo Rate + 
Spread
Effective Rate of 
Interest
Baroda Home 
Loan to NonStaff members
For Salaried* BRLLR+1.00% 
to BRLLR + 
2.35%
From 10.15% to 
11.50%
For NonSalaried*
BRLLR+1.10% 
to 
BRLLR+2.45%
From 10.25% to 
11.60%
Processing Charges :
1. For Takeover leads received through Non-DSA (Digital or Non-digital) –
NIL, subject to Login Fee of Rs.1,500/- which is to be refunded on first 
disbursement.
2. For Home Loan in approved projects (Digital or Non-digital):
100% waiver in Processing charges, subject to recovery of out-of-pocket 
expenses of ₹3,500.00 + GST (Per Property)./p>
3. Under Home Loan Scheme for Central/State Govt/PSUs employees – for 
non-DSA (including leads generated by DST) –
NIL, subject to Login Fee of Rs.1,500/- which is to be refunded on first 
disbursement.
4. For Other Home Loan lead sourced through Non-Digital Channel:
100% waiver in Processing charges, subject to recovery of out-of-pocket 
expenses of ₹10,000.00 + GST (Per Property)
5. For Home Loan leads sourced through Digital Channel:
100% waiver in Processing charges, subject to recovery of out-of-pocket 
expenses of ₹5,000.00 + GST (Per Property) to ₹8,500.00 + GST (conditions 
apply)
Documents Required
Most Important Terms and Conditions:
All Documents Self –Attested By Applicant /Co -Applicant /Guarantor
A) RESIDENT INDIAN:
I.
1. Duly Filled and Signed Application form along with three Photograph s
2. Proof of Identity - PAN CARD (mandatory for loan application above Rs.10.00 lacs
3. Driver’s License / Voter ID / Passport / Aadhar Card
4. Proof of Residence (Driver’s License / Ration Card/ Voter ID / Passport / Aadhar Card
/ Registered Rent Agreement)
5. If any previous loan then loan a/c statement for last 1 year along with sanction letter.
( If loan from BoB, Account No / Cust Id required)
6. Proof of assets held like LIC/NSC/KVP/MF/PROPERTY
7. Assets and Liabilities statement
8. ITR Verification report
9. Additional Do cuments for (Applicants /Co - applicants whose income to be considered for eligibility)
Salaried Individuals Self-Employed Individuals / Professionals /Others Farmers /agriculturist
a. Latest 3 months Salary Slips & latest 1 month salary slip for
Guara ntors
Balance Sheet & Profit & Loss A/c, Computation of Income - Last 
2 years
Talati’s/Gram Sevak/Village Revenue officer Certificate for previous two 
years income and Mamladaar’s /Block Revenue officer certificate for last 
year’s
income.
b. Form 16 & ITR – last 1 years of Applicants &
Guarantors(if any )
Income Tax Returns – last 2 years for Applicants, 26 AS ,
Traces
c. Copy of Employee identity Card provided by Employer Business proof : Gomasta License, Registration 
Certificate, Service Tax
Registration, etc
Land revenue records – Form 6, 7/12, 8A ;
d. Appointment
/Confirmation / Promotion /Increment letter evidencing
duration of employment
IT Assessment / Clearance Certificate, Income Tax Challans / TDS 
Certificate (Form 16A) / Form 26 AS for
income d eclared in ITR.
e. 6 months Bank A/c statement (Salary / Individual) or 
Account No if account is with BoB.
In Case of Applicants engaged in Business through
Partnership Firm/Private Ltd. Company:
12 months Bank A/c statement (Individual)
f. i. Firm’s PAN c ard, Address Proof of Firm /
 Company
Target Group
 Resident Indians
 Non-resident Indians (NRIs) holding an Indian passport or Persons of 
Indian origin (PIOs) holding a foreign passport or overseas citizens of 
India (OCI).
 Staff members (availing under public scheme)
Eligibility of Borrower(s)
 Individuals - singly or jointly.
 HUFs are not eligible.
Resident Indian
Applicant/co-applicant(s), whose income are considered for eligibility, should 
be employed/engaged in business/profession for a minimum period of 1 year 
(for salaried) and/or 2 years (for non-salaried).
Break in service, if any, can be allowed up to a maximum period of 3 months.
NRI/PIO/OCI
 Applicant/co-applicant(s), whose income are considered for eligibility, 
should be having a regular job abroad in a reputed Indian/foreign 
company, organization or government department holding a valid job 
contract/work permit for the minimum past 2 years.
OR
Should be employed/self-employed or having a business unit and 
staying abroad at least for 2 years.
 Applicant/co-applicant(s), whose income is considered for eligibility, 
should have minimum gross annual income equivalent to Rs. 5 lakhs 
per annum. If the applicant/co-applicant(s), whose income is considered 
for eligibility includes NRI, a minimum Gross Annual Income of Rs. 5 
lakhs (income of applicant/co-applicant together) can be considered for 
this criteria.
Person of Indian Origin (PIO) is specified as under
A citizen of any country other than Bangladesh/Pakistan/Sri 
Lanka/Afghanistan/China/Iran/Nepal/Bhutan if –
 He at any time held Indian passport or
 He or either of his parents or any of his grandparents was a citizen of 
India by virtue of the constitution of India of the Citizenship Act 1955, or
 The person is a spouse of an Indian citizen or a person referred to in 
sub-clause (a) or (b) above.
Overseas Citizens of India (OCI) is specified as under
 A person registered as Overseas Citizen of India (OCI) under section 7 A 
of the Citizenship Act, 1955.
 As per the guidelines, an Overseas Citizens of India (OCI) has to be 
necessarily a Person of India Origin (PIO).
 A foreign national, who was eligible to become citizen of India on 26/ 01/ 
1950 or was a citizen of India on or at any time after 26/ 01/ 1950 or 
belonged to a territory that became part of India after 15/08/1947, and 
his/her children and grandchildren, provided his/her country of 
citizenship allows dual citizenship in some form or other under the local 
laws, is eligible for registration as Overseas Citizen of India (OCI). Minor 
children of this person are also eligible for OCI. However, if the applicant 
had ever been a citizen of Pakistan or Bangladesh, he/she will not be 
eligible for OCI.
Co-Applicants
 The close relatives of the applicant can be added as a co-applicant for 
higher eligibility.
 If the applicant wants to add any person who is not a close relative as a 
co-applicant, the same can be considered only if he/she is the joint 
owner of property.
List of Close Relatives
Spouse, father, mother (including step mother), son (including step son), son’s 
wife, daughter (including step daughter), daughter’s husband, brother/sister 
(including stepbrother/sister), brother’s wife, sister (including step sister) of 
spouse, sister’s husband, brother (including step brother) of spouse.
Maximum Loan amount
 Mumbai : Rs. 20 Crores
 Hyderabad, New Delhi and Bengaluru : Rs. 7.50 Crores
 Other Metros : Rs. 5.00 Crores
o Urban Areas : Rs. 3.00 Crores
 Semi-urban and Rural : Rs. 1.00 Crores
 Chandigarh, Panchkula & Mohali:- Rs. 5 Crores
Repayment Period
 Maximum period of loans shall be 30 years initially, including the 
maximum moratorium period of 36 months.
 Maximum moratorium period shall be 36 months as under.
 18 month moratorium period for under construction houses and 
building up to 7th floor and thereafter 6 months additional moratorium 
per floor subject to a maximum moratorium of 36 months.
Repayment Capacity
Total deductions including proposed EMI should not exceed as follows -
Salaried Persons
 Net Monthly income less than Rs.25,000: 40%
 Net monthly income Rs.25,000 and above but less than Rs.50,000: 60%
 Net monthly income Rs.50,000 and above but less than Rs.1.00 lakhs: 
65%
 Net monthly income Rs.1.00 lakhs and above but less than Rs.2.00 
lakhs: 70%
 Net monthly income Rs.2.00 lakhs and above: 75%
Others
 Average Net Annual Income (for last 2 years) up to Rs.6 lakhs: 70%
 Average Net Annual Income (for last 2 years) more than Rs.6 lakhs: 80%
Margin Norms & Loan to Value (LTV) Ratio
Loan Amount Margin LTV Ratio
Loans up to Rs.30/- Lacs 10% 90%
Loans above Rs.30/- Lacs up to Rs.75/- Lacs 20% 80%
Loans above Rs.75/- Lacs 25% 75%
Age
 Minimum: Borrower - 21 years, Co-applicant - 18 years
 Maximum: Maximum age can be considered up to 70 years*
Security
 Mortgage of the property constructed/purchased or
 If mortgage is not feasible, the Bank at its discretion can accept security 
in the form of insurance policies, government promissory notes, shares 
and debentures, gold ornaments, etc.
Repayment
 Maximum period of loans shall be 30 years initially, including the 
maximum moratorium period of 36 months.
 Maximum moratorium period shall be 36 months as under.
 18 month moratorium period for under construction houses and 
building up to 7th floor and thereafter 6 months additional moratorium 
per floor subject to a maximum moratorium of 36 months.
 Loan to be repaid in Equated Monthly Instalments (EMI)
 In case of farmers/agriculturists, repayment can be allowed in Half 
Yearly Instalments coinciding with harvesting/marketing of major crops 
produced.
 Recovery of interest for the moratorium period Interest charged during 
the moratorium period is to be recovered as and when debited.
Pre-Closure Charges
Nil
Group Credit Life Insurance Cover
Bank of Baroda has tie-up arrangements with M/s. IndiaFirst Life Insurance Co. 
for providing Life Insurance Cover to our Home Loan borrowers. This is an 
optional scheme of Life Insurance and is at the cost of borrower which may be 
funded by the Bank at the time of sanction and is recoverable with the EMIs of 
the Loan.
The salient Features of the Group Credit Life Policy offered by M/s. IndiaFirst 
Life Insurance Co. as under:
 It is an optional scheme for the benefit of Home Loan borrowers and the 
borrower will be given an option by the branches to choose the 
insurance provider between IndiaFirst Life Insurance as per his choice.
 It is a group insurance scheme, which protects against unforeseen 
consequences of death of borrower.
 Family of the borrower is not required to repay the loan to the extent of 
claim settled amount, in case of death of the borrower.
 Outstanding cover amount due, will be paid by insurance provider as 
per cover schedule.
 Slippage of the account into NPA category due to death of the borrower 
can be avoided.
 Life cover is available against payment of one time premium and amount 
of premium is based on the age of the borrower, amount of loan, interest 
rate and tenure of the loan.
 Premium amount for obtaining the cover can be financed as part of loan 
at the request of borrower and accordingly EMI will be calculated. But in 
case of Home Loans, guidelines regarding compliance of LTV Ratio 
must be strictly ensured.
 Premium amount may be part of the loan as per total eligibility of the 
borrower may be funded by the Bank at the time of sanction and is 
recoverable with the EMIs of the loan.
 In case of foreclosure of loan, certain portion of premium paid shall be 
refunded by the insurance provider'''

In [2]:
from summarys import Summarization

c:\Users\DudukuntaSashidharRe\AppData\Local\Programs\Python\Python310\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
import os
import time
# Set an environment variable
os.environ['PINECONE_API_KEY'] = '193bfd5b-1a4a-4c8b-bc06-7ec7c4cfc66a'

PINECONE_API_KEY="193bfd5b-1a4a-4c8b-bc06-7ec7c4cfc66a"

In [7]:
import re
import uuid
from openai import AzureOpenAI
from pinecone import Pinecone, ServerlessSpec

In [8]:
gpt_engine_name="gpt-35-turbo-16k"
embedding_model_name="text-embedding-002"
azure_endpoint = "https://securityappdewa.openai.azure.com"
api_key="576b185ff87d4f50962560d349d06332"
api_version= "2023-07-01-preview"

In [9]:
s=Summarization('indexdb', PINECONE_API_KEY, gpt_engine_name, embedding_model_name, api_key, azure_endpoint, api_version, "azure_openai")

In [10]:
su=await s.summarize_document(text)

In [11]:
print(su)

The document provides information about the Bank of Baroda Home Loan, including its features, benefits, eligibility criteria, interest rates, charges, and required documents. Here are the key points from the document:

1. Bank of Baroda Home Loan offers several exclusive features and benefits for aspiring homeowners.
2. The loan can be used for various purposes such as buying a plot, purchasing a flat, building a home, or extending an existing residence.
3. The loan amount is determined based on the location and income of the applicants.
4. The interest rates are linked to the Baroda Repo Linked Lending Rate (BRLLR) of the bank and are reset monthly.
5. The loan repayment period can be up to 30 years, with a moratorium period of up to 36 months.
6. Collateral for the loan can be in the form of the constructed or purchased property, insurance policies, government promissory notes, shares and debentures, gold ornaments, etc.
7. The eligibility criteria include resident Indians, non-resid